In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
from torchvision import datasets,transforms
from glob import glob
import os
from PIL import Image
from matplotlib import patches
from torch.utils.data import Dataset
import torchvision
from torch.utils.data import DataLoader

In [13]:
train_dir ='/content/drive/MyDrive/global_wheat/train/'
test_dir = '/content/drive/MyDrive/global_wheat/test/'
train = pd.read_csv( '/content/drive/MyDrive/global_wheat/train.csv' )

In [14]:
train_images = glob(train_dir +'*')
test_images = glob(test_dir +'*')

In [15]:
train_images[:10]

['/content/drive/MyDrive/global_wheat/train/b1642dc6f.jpg',
 '/content/drive/MyDrive/global_wheat/train/b61ff5016.jpg',
 '/content/drive/MyDrive/global_wheat/train/b4661eb6f.jpg',
 '/content/drive/MyDrive/global_wheat/train/afe608129.jpg',
 '/content/drive/MyDrive/global_wheat/train/b2efe29e7.jpg',
 '/content/drive/MyDrive/global_wheat/train/b49e3593d.jpg',
 '/content/drive/MyDrive/global_wheat/train/b00b900a1.jpg',
 '/content/drive/MyDrive/global_wheat/train/b24a3ecee.jpg',
 '/content/drive/MyDrive/global_wheat/train/afd82a1d7.jpg',
 '/content/drive/MyDrive/global_wheat/train/b13f38e6f.jpg']

In [16]:
train.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [17]:
train['bbox'] = train['bbox'].fillna('[0,0,0,0]')
bbox_items = train['bbox'].str.split(',',expand = True)
train['x'] = bbox_items[0].str.strip('[').astype(float)
train['y'] = bbox_items[1].str.strip(' ').astype(float)
train['w'] = bbox_items[2].str.strip(' ').astype(float)
train['h'] = bbox_items[3].str.strip(']').astype(float)
train.head()

,image_id,width,height,bbox,source,x,y,w,h
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,124.0,117.0


In [18]:
img_id = train['image_id'].unique()
valid_ids = img_id[-665:]
train_ids = img_id[:-665]

In [19]:
valid_df = train[train['image_id'].isin(valid_ids)]
train_df = train[train['image_id'].isin(train_ids)]

In [20]:
valid_df.shape,train_df.shape
train['area'] = train['w']*train['h']

In [21]:
train

,image_id,width,height,bbox,source,x,y,w,h,area
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1,834.0,222.0,56.0,36.0,2016.0
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1,226.0,548.0,130.0,58.0,7540.0
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1,377.0,504.0,74.0,160.0,11840.0
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1,834.0,95.0,109.0,107.0,11663.0
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1,26.0,144.0,124.0,117.0,14508.0
...,...,...,...,...,...,...,...,...,...,...
147788,5e0747034,1024,1024,"[64.0, 619.0, 84.0, 95.0]",arvalis_2,64.0,619.0,84.0,95.0,7980.0
147789,5e0747034,1024,1024,"[292.0, 549.0, 107.0, 82.0]",arvalis_2,292.0,549.0,107.0,82.0,8774.0
147790,5e0747034,1024,1024,"[134.0, 228.0, 141.0, 71.0]",arvalis_2,134.0,228.0,141.0,71.0,10011.0
147791,5e0747034,1024,1024,"[430.0, 13.0, 184.0, 79.0]",arvalis_2,430.0,13.0,184.0,79.0,14536.0


In [22]:
class Wheatdatasets(Dataset):
    
    def __init__(self,dataframe,image_dir,transforms = None):
        
        super().__init__()
    
        self.image_id = dataframe['image_id'].unique()
        self.df = dataframe
        self.img_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self,index:int):
        
        image_id = self.image_id[index]
        record = self.df[self.df['image_id']==image_id]
        
        image = cv2.imread(self.img_dir+image_id+'.jpg',cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB).astype(np.float32)
        image = image/255.0
       
        if self.transforms:
            sample ={
                'image':image
            }
            sample =self.transforms(**sample)
            image = sample['image']
            
            return image,image_id
        
    def __len__(self) -> int:
        return self.image_id.shape[0]

In [ ]:
def get_transform():
    return al.Compose([
        ToTensorV2(p=1.0)
    ])

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = False,pretrained_backbone=False)